# Google form analysis

Analysis of results extracted from Google forms in csv format.

## Preparation

In [8]:
%run "../Utilities/Preparation.ipynb"
%run "../Static data/Google form correct answers.ipynb"

# Constants

In [9]:
gformPath = "../../data/Google forms/"
genericFormName = '2017-11-02-gform-'
enSuffix = 'en'
frSuffix = 'fr'
csvSuffix = '.csv'
gformEN = pd.read_csv(gformPath + genericFormName + enSuffix + csvSuffix)
gformFR = pd.read_csv(gformPath + genericFormName + frSuffix + csvSuffix)

### Temporary, while gformEN and gformFR are not merged

In [10]:
#gform = pd.concat([gformEN, gformFR])
gform = gformEN

In [11]:
gformEN.head(2)

,Timestamp,Are you interested in video games?,Do you play video games?,How old are you?,What is your gender?,How long have you studied biology?,Are you interested in biology?,"Before playing Hero.Coli, had you ever heard about synthetic biology?","Before playing Hero.Coli, had you ever heard about BioBricks?",Have you ever played an older version of Hero.Coli before?,...,What does this device do?.5,What does this device do?.6,What does this device do?.7,"Guess: what would a device producing l-arabinose do, if it started with a l-arabinose-induced promoter?",Guess: the bacterium would glow yellow...,What is the species of the bacterium of the game?,What is the scientific name of the tails of the bacterium?,Find the antibiotic:,You can write down remarks here.,Do not edit - pre-filled anonymous ID
0,2017/08/18 3:22:26 PM GMT+1,Extremely,Extremely,23,Female,Until bachelor's degree,Extremely,Yes,NaN,No,...,It makes it possible to move faster,It generates antibiotic resistance,It generates green fluorescence in presence of...,"After being induced, it would produce more and...",I don't know,E. Coli,Flagella,Ampicillin,Your game is amazing. One suggestion. In sever...,8d352896-a3f1-471c-8439-0f426df901c1
1,2017/09/06 1:28:03 PM GMT+1,Moderately,Moderately,28,Other,Until the end of high school,Moderately,Yes,NaN,No,...,It makes it possible to move faster,It generates antibiotic resistance,It generates green fluorescence in presence of...,"After being induced, it would produce more and...",I don't know,E. Coli,Flagella,Ampicillin,"This was a nice educational game, thank you fo...",7037c5b2-c286-498e-9784-9a061c778609


In [12]:
#localplayerguidkey = 'Ne pas modifier - identifiant anonyme prérempli'
localplayerguidkey = 'Do not edit -  pre-filled anonymous ID'
localplayerguidindex = gformEN.columns.get_loc(localplayerguidkey)
localplayerguidindex

41

In [13]:
firstEvaluationQuestionKey = 'In order to modify the abilities of the bacterium, you have to...'
firstEvaluationQuestionIndex = gformEN.columns.get_loc(firstEvaluationQuestionKey)
firstEvaluationQuestionIndex

13

# Functions
<a id=functions />

In [14]:
#user id in format localplayerguid = '"8d352896-a3f1-471c-8439-0f426df901c1"'

def getAllResponders():
    userIds = gform[localplayerguidkey].values
    result = []
    for id in userIds:
        result.append('"' + id + '"')
    return result

def hasAnswered( userId ):
    return userId.replace('"','') in gform[localplayerguidkey].values

def getAnswers( userId):
    answers = gform[gform[localplayerguidkey]==userId.replace('"','')]
    _columnAnswers = answers.T
    _columnAnswers.columns = ['answers']
    return _columnAnswers

def getCorrections( userId):
    _columnAnswers = getAnswers( userId )
    
    _columnAnswers['correct'] = _columnAnswers['answers']
    
    # demographic questions
    for index in range(0, firstEvaluationQuestionIndex):
        _columnAnswers['correct'][index] = 'NaN'
        
    # evaluation questions
    for index in range(firstEvaluationQuestionIndex, len(_columnAnswers.values)-2):
        _columnAnswers['correct'][index] = str(_columnAnswers['answers'][index]).startswith(str(correctAnswers['correctAnswer'][index]))
    
    # free opinion question
    _columnAnswers['correct'][len(_columnAnswers.values)-2] = 'NaN'
    # pre-filled ID question
    _columnAnswers['correct'][len(_columnAnswers.values)-1] = 'NaN'
    return _columnAnswers

#function that returns the score from user id
def getScore( userId ):
    if hasAnswered( userId ):
        _columnAnswers = getCorrections( userId)
        score = (_columnAnswers['correct']).value_counts()[True]
        return score
    else:
        return np.NaN

#function that returns the list of checkpoints from user id
def getValidatedCheckpoints( userId ):
    _columnAnswers = getCorrections( userId)
    questionnaireValidatedCheckpointsPerQuestion = pd.Series(np.nan, index=range(len(checkpointQuestionMatching)))
    
    for index in range(0, len(questionnaireValidatedCheckpointsPerQuestion)):
        if _columnAnswers['correct'][index]==True:
            questionnaireValidatedCheckpointsPerQuestion[index] = checkpointQuestionMatching['checkpoint'][index]
        else:
            questionnaireValidatedCheckpointsPerQuestion[index] = ''
    
    questionnaireValidatedCheckpoints = questionnaireValidatedCheckpointsPerQuestion.unique()
    questionnaireValidatedCheckpoints = questionnaireValidatedCheckpoints[questionnaireValidatedCheckpoints!='']
    questionnaireValidatedCheckpoints = pd.Series(questionnaireValidatedCheckpoints)
    questionnaireValidatedCheckpoints = questionnaireValidatedCheckpoints.sort_values()
    questionnaireValidatedCheckpoints.index = range(0, len(questionnaireValidatedCheckpoints))
    return questionnaireValidatedCheckpoints

def getNonValidated( checkpoints ):
    return pd.Series(np.setdiff1d(checkpointQuestionMatching['checkpoint'].values, checkpoints.values))

def getNonValidatedCheckpoints( userId ):
    validated = getValidatedCheckpoints( userId )
    return getNonValidated(validated)